In [2]:
import pandas as pd

In [3]:
data_dir = r"../data/"
global_sqm_data_name = r"global_property_guide-global_sqm_prices_by_country_01-11-2022.csv"

date_of_data = global_sqm_data_name.split('_')[-1][:-4]
data_source_url = r'https://www.globalpropertyguide.com/most-expensive-cities'

In [4]:
df = pd.read_csv(data_dir + global_sqm_data_name)
df.head()

,Country/City,Buying Price\nUS $ per Sq. M.,Price/Rent\nRatio (x),Rent per\nMonth ($ or €),Gross\nRental Yield
0,Hong Kong,"$ 23,695",36x,"$ 2,149",2.78%
1,Singapore,"$ 16,120",21x,"$ 5,075",4.74%
2,United Kingdom,"$ 15,125",16x,"€1,999",6.21%
3,France,"$ 14,808",25x,"€1,441",4.06%
4,Israel,"$ 13,820",30x,"$ 1,615",3.30%


In [5]:
df = df.drop(columns=['Price/Rent\nRatio (x)', 'Gross\nRental Yield'])
df.head()

,Country/City,Buying Price\nUS $ per Sq. M.,Rent per\nMonth ($ or €)
0,Hong Kong,"$ 23,695","$ 2,149"
1,Singapore,"$ 16,120","$ 5,075"
2,United Kingdom,"$ 15,125","€1,999"
3,France,"$ 14,808","€1,441"
4,Israel,"$ 13,820","$ 1,615"


In [6]:
df = df.rename(columns={'Country/City': 'country', 'Buying Price\nUS $ per Sq. M.': 'purchase_price_per_sqm_usd', 'Rent per\nMonth ($ or €)': 'rent_per_month'})
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month
0,Hong Kong,"$ 23,695","$ 2,149"
1,Singapore,"$ 16,120","$ 5,075"
2,United Kingdom,"$ 15,125","€1,999"
3,France,"$ 14,808","€1,441"
4,Israel,"$ 13,820","$ 1,615"


In [7]:
df['data_source_url'] = data_source_url
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month,data_source_url
0,Hong Kong,"$ 23,695","$ 2,149",https://www.globalpropertyguide.com/most-expen...
1,Singapore,"$ 16,120","$ 5,075",https://www.globalpropertyguide.com/most-expen...
2,United Kingdom,"$ 15,125","€1,999",https://www.globalpropertyguide.com/most-expen...
3,France,"$ 14,808","€1,441",https://www.globalpropertyguide.com/most-expen...
4,Israel,"$ 13,820","$ 1,615",https://www.globalpropertyguide.com/most-expen...


In [8]:
df['date_source_data'] = date_of_data
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month,data_source_url,date_source_data
0,Hong Kong,"$ 23,695","$ 2,149",https://www.globalpropertyguide.com/most-expen...,01-11-2022
1,Singapore,"$ 16,120","$ 5,075",https://www.globalpropertyguide.com/most-expen...,01-11-2022
2,United Kingdom,"$ 15,125","€1,999",https://www.globalpropertyguide.com/most-expen...,01-11-2022
3,France,"$ 14,808","€1,441",https://www.globalpropertyguide.com/most-expen...,01-11-2022
4,Israel,"$ 13,820","$ 1,615",https://www.globalpropertyguide.com/most-expen...,01-11-2022


In [9]:
usd_to_euro_conversion = 1.0098

def convert_str_price_to_int(val):
    return int(''.join(val[1:].split(',')))

def convert_to_int_usd(val):
    if val[0] == '$':
        return round(convert_str_price_to_int(val))
    elif val[0] == '€':
        return round(convert_str_price_to_int(val) * usd_to_euro_conversion)
    else:
        raise ValueError("Price is neither USD or EURO", val)
    
def get_usd_value_for_row_element(row, column_name):
    try:
        result = convert_to_int_usd(row[column_name])
        return result
    except ValueError as e:
        print(f"Exception occurred at element {column_name} in row: {row}")
        print("Error:", e)
        return None  # Return a value or indicator of failure

In [10]:
df['rent_per_month'] = df.apply(get_usd_value_for_row_element, args=['rent_per_month'], axis=1)
df['purchase_price_per_sqm_usd'] = df.apply(get_usd_value_for_row_element, args=['purchase_price_per_sqm_usd'], axis=1)

Exception occurred at element rent_per_month in row: country                                                                 Finland
purchase_price_per_sqm_usd                                              $ 5,874
rent_per_month                                                             n.a.
data_source_url               https://www.globalpropertyguide.com/most-expen...
date_source_data                                                     01-11-2022
Name: 18, dtype: object
Error: ('Price is neither USD or EURO', 'n.a.')
Exception occurred at element rent_per_month in row: country                                                             Puerto Rico
purchase_price_per_sqm_usd                                              $ 2,700
rent_per_month                                                             n.a.
data_source_url               https://www.globalpropertyguide.com/most-expen...
date_source_data                                                     01-11-2022
Name: 37, dtype: objec

There appears to be some missing values in the `rent_per_month` column for Finland and Puerto Rico, there will need to be stored as `null` in the database.

In [11]:
df = df.rename(columns={'rent_per_month': 'rent_per_month_usd'})
df['rent_per_month_usd'] = df['rent_per_month_usd'].astype(pd.Int64Dtype())
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data
0,Hong Kong,23695,2149,https://www.globalpropertyguide.com/most-expen...,01-11-2022
1,Singapore,16120,5075,https://www.globalpropertyguide.com/most-expen...,01-11-2022
2,United Kingdom,15125,2019,https://www.globalpropertyguide.com/most-expen...,01-11-2022
3,France,14808,1455,https://www.globalpropertyguide.com/most-expen...,01-11-2022
4,Israel,13820,1615,https://www.globalpropertyguide.com/most-expen...,01-11-2022


In [12]:
from datetime import datetime

def convert_to_date(date_string):
    return datetime.strptime(date_string, '%d-%m-%Y')

convert_to_date('01-11-2022')

datetime.datetime(2022, 11, 1, 0, 0)

In [13]:
df['date_source_data'] = df['date_source_data'].map(convert_to_date)
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data
0,Hong Kong,23695,2149,https://www.globalpropertyguide.com/most-expen...,2022-11-01
1,Singapore,16120,5075,https://www.globalpropertyguide.com/most-expen...,2022-11-01
2,United Kingdom,15125,2019,https://www.globalpropertyguide.com/most-expen...,2022-11-01
3,France,14808,1455,https://www.globalpropertyguide.com/most-expen...,2022-11-01
4,Israel,13820,1615,https://www.globalpropertyguide.com/most-expen...,2022-11-01


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   country                     59 non-null     object        
 1   purchase_price_per_sqm_usd  59 non-null     int64         
 2   rent_per_month_usd          57 non-null     Int64         
 3   data_source_url             59 non-null     object        
 4   date_source_data            59 non-null     datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), int64(1), object(2)
memory usage: 2.5+ KB


## Capital city mapping

In [15]:
import csv

capital_city_file_name = "country_capital_city.csv"

with open(data_dir + capital_city_file_name, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    country_capital_dict = {row['Country']:row['Capital City'] for row in csv_reader}

next(iter(country_capital_dict.items()))

('Afghanistan', 'Kabul')

In [16]:
df['city'] = df['country'].map(country_capital_dict)
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data,city
0,Hong Kong,23695,2149,https://www.globalpropertyguide.com/most-expen...,2022-11-01,NaN
1,Singapore,16120,5075,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Singapore
2,United Kingdom,15125,2019,https://www.globalpropertyguide.com/most-expen...,2022-11-01,London
3,France,14808,1455,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Paris
4,Israel,13820,1615,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Jerusalem (very limited international recognit...


##### Check if values have been mapped correctly

In [17]:
df[df['city'].isna()]

,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data,city
0,Hong Kong,23695,2149,https://www.globalpropertyguide.com/most-expen...,2022-11-01,NaN
5,Taiwan,10955,984,https://www.globalpropertyguide.com/most-expen...,2022-11-01,NaN
22,Czech Republic,5227,937,https://www.globalpropertyguide.com/most-expen...,2022-11-01,NaN
37,Puerto Rico,2700,<NA>,https://www.globalpropertyguide.com/most-expen...,2022-11-01,NaN
46,Turkey,1955,422,https://www.globalpropertyguide.com/most-expen...,2022-11-01,NaN


We can see here that five countries have failed to map. After analysing the country_capital data this due to various reasons (e.g. Country not recognised and hence no capital, difference in spelling).

Due to the small amount of issues I will manually add these capital names.

In [18]:
df.loc[df['country'] == 'Hong Kong', 'city'] = 'Hong Kong'
df.loc[df['country'] == 'Taiwan', 'city'] = 'Taipei'
df.loc[df['country'] == 'Puerto Rico', 'city'] = 'San Juan'
df.loc[df['country'] == 'Czech Republic', 'city'] = 'Prague'
df.loc[df['country'] == 'Turkey', 'city'] = 'Ankara'

In [19]:
df[df['city'].isna()]

,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data,city


In [20]:
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data,city
0,Hong Kong,23695,2149,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Hong Kong
1,Singapore,16120,5075,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Singapore
2,United Kingdom,15125,2019,https://www.globalpropertyguide.com/most-expen...,2022-11-01,London
3,France,14808,1455,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Paris
4,Israel,13820,1615,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Jerusalem (very limited international recognit...


In [21]:
df['city']

0                                             Hong Kong
1                                             Singapore
2                                                London
3                                                 Paris
4     Jerusalem (very limited international recognit...
5                                                Taipei
6                                            Luxembourg
7                                                  Bern
8                                              Canberra
9                                         Amsterdam[14]
10                                                 Oslo
11                                            Stockholm
12                                                Tokyo
13                                           Copenhagen
14                                                 Rome
15                                               Dublin
16                                           Wellington
17                                              

There's quite a few capital city entries that have added info contained in either parenthesis or square brackets. I don't want this added info so will remove.

In [22]:
import re

def find_brackets(input_str):
    brackets = ['[', ']', '(', ')']
    return [bracket for bracket in brackets if bracket in input_str] 

def remove_brackets(input_str, brackets_to_remove):
    # Create a pattern to match any of the characters
    pattern = '|'.join(re.escape(bracket) for bracket in brackets_to_remove)
    
    # Split the string using the pattern
    parts = re.split(pattern, input_str)

    # Remove parts that follow a bracket
    return parts[0]

def clean_cities(input_str):
    brackets_to_remove = find_brackets(input_str)
    if len(brackets_to_remove) == 0:
        result = input_str
    else:
        result = remove_brackets(input_str, brackets_to_remove).rstrip()
        
    return result

print(clean_cities('Jerusalem (very limited international recognit...)'))
print(clean_cities('Kuala Lumpur[12]'))
print(clean_cities('Jakarta[9]'))
print(clean_cities('London'))

Jerusalem
Kuala Lumpur
Jakarta
London


In [23]:
df['city'] = df['city'].map(clean_cities)
df.head()

,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data,city
0,Hong Kong,23695,2149,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Hong Kong
1,Singapore,16120,5075,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Singapore
2,United Kingdom,15125,2019,https://www.globalpropertyguide.com/most-expen...,2022-11-01,London
3,France,14808,1455,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Paris
4,Israel,13820,1615,https://www.globalpropertyguide.com/most-expen...,2022-11-01,Jerusalem


In [24]:
df = pd.concat([df['city'], df.drop('city', axis=1)], axis=1)

In [25]:
df.head()

,city,country,purchase_price_per_sqm_usd,rent_per_month_usd,data_source_url,date_source_data
0,Hong Kong,Hong Kong,23695,2149,https://www.globalpropertyguide.com/most-expen...,2022-11-01
1,Singapore,Singapore,16120,5075,https://www.globalpropertyguide.com/most-expen...,2022-11-01
2,London,United Kingdom,15125,2019,https://www.globalpropertyguide.com/most-expen...,2022-11-01
3,Paris,France,14808,1455,https://www.globalpropertyguide.com/most-expen...,2022-11-01
4,Jerusalem,Israel,13820,1615,https://www.globalpropertyguide.com/most-expen...,2022-11-01


In [ ]:
##